In [27]:
"""
@misc{home-credit-credit-risk-model-stability,
    author = {Daniel Herman, Tomas Jelinek, Walter Reade, Maggie Demkin, Addison Howard},
    title = {Home Credit - Credit Risk Model Stability},
    publisher = {Kaggle},
    year = {2024},
    url = {https://kaggle.com/competitions/home-credit-credit-risk-model-stability}
}
"""

'\n@misc{home-credit-credit-risk-model-stability,\n    author = {Daniel Herman, Tomas Jelinek, Walter Reade, Maggie Demkin, Addison Howard},\n    title = {Home Credit - Credit Risk Model Stability},\n    publisher = {Kaggle},\n    year = {2024},\n    url = {https://kaggle.com/competitions/home-credit-credit-risk-model-stability}\n}\n'

In [28]:
"""
Home Credit - Credit Risk Model Stability
Create a model measured against feature stability over time
The goal of this competition is to predict which clients are more likely to default on their loans. 
The evaluation will favor solutions that are stable over time.
"""

'\nHome Credit - Credit Risk Model Stability\nCreate a model measured against feature stability over time\nThe goal of this competition is to predict which clients are more likely to default on their loans. \nThe evaluation will favor solutions that are stable over time.\n'

In [29]:
"""
Lack of credit history could indicate youth or a preference for using cash.
Traditional data scarcity often leads to loan application rejections.
Data science is essential in determining loan repayment capabilities, potentially making loans more accessible to those in need.
Consumer finance providers utilize various statistical and machine learning methods in scorecards to assess loan risk.
Scorecards must be updated regularly due to the constant change in client behavior, making the process time-consuming.
The stability of a scorecard is crucial; a performance decline could mean issuing loans to higher-risk clients.
Early detection of loan repayment issues is challenging and usually visible only after loan due dates.
Balancing the stability and performance of scorecards is necessary before their implementation.
Home Credit, established in 1997, aims to offer responsible lending to people without significant credit history.
Home Credit's efforts in financial inclusion have previously been showcased in a Kaggle competition.
Improving scorecard assessments could help consumer finance providers accept more loan applications, enhancing financial access for historically underserved groups.
"""

"\nLack of credit history could indicate youth or a preference for using cash.\nTraditional data scarcity often leads to loan application rejections.\nData science is essential in determining loan repayment capabilities, potentially making loans more accessible to those in need.\nConsumer finance providers utilize various statistical and machine learning methods in scorecards to assess loan risk.\nScorecards must be updated regularly due to the constant change in client behavior, making the process time-consuming.\nThe stability of a scorecard is crucial; a performance decline could mean issuing loans to higher-risk clients.\nEarly detection of loan repayment issues is challenging and usually visible only after loan due dates.\nBalancing the stability and performance of scorecards is necessary before their implementation.\nHome Credit, established in 1997, aims to offer responsible lending to people without significant credit history.\nHome Credit's efforts in financial inclusion have 

In [30]:
"""
For each case_id in the test set, you must predict a probability for the target score. The file should contain a header and have the following format:

case_id,score
57543,0.1
57544,0.9
57545,0.5
etc.


Submission file must be named submission.csv

"""

'\nFor each case_id in the test set, you must predict a probability for the target score. The file should contain a header and have the following format:\n\ncase_id,score\n57543,0.1\n57544,0.9\n57545,0.5\netc.\n\n\nSubmission file must be named submission.csv\n\n'

In [31]:
"""
 predicting default of clients based on internal and external information that are available for each client. Scoring is 
 performed using custom metric that not only evaluates the AUC of predictions but also considers the stability of predictions 
 model across the data range of the test set.
"""

'\n predicting default of clients based on internal and external information that are available for each client. Scoring is \n performed using custom metric that not only evaluates the AUC of predictions but also considers the stability of predictions \n model across the data range of the test set.\n'

In [32]:
import polars as pl #Libreria para manipulacion de datos, fast wrote in rust https://docs.pola.rs 
import numpy as np #Libreria para manipulacion de datos 
import pandas as pd #Libreria para manipulacion de datos 
import lightgbm as lgb #Libreria para modelos de machine learning,  is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 



In [33]:
#Se cambia la ruta de acuerdo a la ubicacion de los datos
dataPath = "C:/Users/diego/Documents/UP/Negocios/DataProyecto/"

Cargar datos

In [34]:
def establecer_tipos_de_datos_tabla(df: pl.DataFrame) -> pl.DataFrame:
    # implementar aquí todos los tipos de datos deseados para las tablas
    # lo siguiente es solo un ejemplo
    for columna in df.columns:
        # la última letra del nombre de la columna te ayudará a determinar el tipo
        if columna[-1] in ("P", "A"):
            df = df.with_columns(pl.col(columna).cast(pl.Float64).alias(columna)) #Pasa las columnas a float64

    return df

In [35]:
def convertir_cadenas(df: pd.DataFrame) -> pd.DataFrame:
    for columna in df.columns:  
        if df[columna].dtype.name in ['object', 'string']:
            df[columna] = df[columna].astype("string").astype('category')
            categorias_actuales = df[columna].cat.categories
            nuevas_categorias = categorias_actuales.to_list() + ["Desconocido"]
            nuevo_tipo = pd.CategoricalDtype(categories=nuevas_categorias, ordered=True)
            df[columna] = df[columna].astype(nuevo_tipo)
            
    return df

In [36]:
ruta_base = dataPath + "csv_files/train/"

# Leer el archivo base de entrenamiento
tabla_base_entrenamiento = pl.read_csv(ruta_base + "train_base.csv")

# Leer y concatenar los archivos estáticos de entrenamiento
archivo_estatico_1 = pl.read_csv(ruta_base + "train_static_0_0.csv").pipe(establecer_tipos_de_datos_tabla)
archivo_estatico_2 = pl.read_csv(ruta_base + "train_static_0_1.csv").pipe(establecer_tipos_de_datos_tabla)
entrenamiento_estatico = pl.concat([archivo_estatico_1, archivo_estatico_2], how="vertical_relaxed")

# Leer otros archivos de entrenamiento
entrenamiento_estatico_cb = pl.read_csv(ruta_base + "train_static_cb_0.csv").pipe(establecer_tipos_de_datos_tabla)
entrenamiento_persona_1 = pl.read_csv(ruta_base + "train_person_1.csv").pipe(establecer_tipos_de_datos_tabla)
entrenamiento_buro_credito_b_2 = pl.read_csv(ruta_base + "train_credit_bureau_b_2.csv").pipe(establecer_tipos_de_datos_tabla)

In [ ]:
ruta_base_prueba = dataPath + "csv_files/test/"

# Leer el archivo base de prueba
tabla_base_prueba = pl.read_csv(ruta_base_prueba + "test_base.csv")

# Leer y concatenar los archivos estáticos de prueba
archivo_estatico_prueba_1 = pl.read_csv(ruta_base_prueba + "test_static_0_0.csv").pipe(establecer_tipos_de_datos_tabla)
archivo_estatico_prueba_2 = pl.read_csv(ruta_base_prueba + "test_static_0_1.csv").pipe(establecer_tipos_de_datos_tabla)
archivo_estatico_prueba_3 = pl.read_csv(ruta_base_prueba + "test_static_0_2.csv").pipe(establecer_tipos_de_datos_tabla)
prueba_estatica = pl.concat([archivo_estatico_prueba_1, archivo_estatico_prueba_2, archivo_estatico_prueba_3], how="vertical_relaxed")

# Leer otros archivos de prueba
prueba_estatica_cb = pl.read_csv(ruta_base_prueba + "test_static_cb_0.csv").pipe(establecer_tipos_de_datos_tabla)
prueba_persona_1 = pl.read_csv(ruta_base_prueba + "test_person_1.csv").pipe(establecer_tipos_de_datos_tabla)
prueba_buro_credito_b_2 = pl.read_csv(ruta_base_prueba + "test_credit_bureau_b_2.csv").pipe(establecer_tipos_de_datos_tabla)

Feature engineering